<a href="https://colab.research.google.com/github/Indian1999/szombatmi/blob/main/DBSCANclustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing, cluster
import scipy
import geopy
import folium

In [2]:
city = "Dubai"
dtf = pd.read_csv("directory.csv")
dtf = dtf[dtf["City"] == city]
dtf = dtf[["City","Street Address", "Longitude", "Latitude"]]
dtf.head()

FileNotFoundError: [Errno 2] No such file or directory: 'directory.csv'

In [ ]:
dtf["Potential"] = np.random.randint(low=3, high = 11, size=len(dtf))
dtf["Staff"] = dtf["Potential"].apply(lambda x: int(np.random.rand()*x) + 1)
dtf["Capacity"] = dtf["Potential"] - dtf["Staff"]
dtf["Cost"] = np.random.choice(["high", "medium", "low"], size=len(dtf), p=[0.4, 0.5, 0.1])
dtf.head()

In [ ]:
#Határozzuk meg a város koordinátáit:
locator = geopy.geocoders.Nominatim(user_agent="MyCoder")
location = locator.geocode(city)
print(location)
location = [location.latitude, location.longitude]
print(location)

In [ ]:
colors = []
for i in dtf["Cost"]:
  if i == "high":
    colors.append("red")
  if i == "medium":
    colors.append("orange")
  if i == "low":
    colors.append("green")
dtf["Color"] = colors
dtf.head()

In [ ]:
map = folium.Map(location=location, tiles="cartodb positron")
dtf.apply(lambda row: folium.CircleMarker(
    location=[row["Latitude"], row["Longitude"]],
    popup=row["Street Address"],
    color=row["Color"],
    fill = True,
    radius = row["Staff"]).add_to(map), axis=1
)
map

In [ ]:
coords = dtf[["Latitude", "Longitude"]]
distortions = []
for i in range(1,11):
  model = cluster.KMeans(n_clusters=i, random_state=0, n_init="auto")
  model.fit(coords)
  distortions.append(model.inertia_)

plt.plot(range(1,11), distortions)
plt.show()

In [ ]:
model = cluster.KMeans(n_clusters = 5, n_init="auto")
dtf["Cluster Kmeans"] = model.fit_predict(coords)
centers = model.cluster_centers_
print(centers)
dtf.head()

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(x="Latitude", y="Longitude", data=dtf,
                palette=sns.color_palette("bright", 5),
                hue = "Cluster Kmeans", legend = "brief", ax=ax
                )
ax.scatter(centers[:,0], centers[:, 1], s = 50, c = "black", marker="x")

In [ ]:
dtf = dtf.sort_values(by = ["Cluster Kmeans"])
dtf.head()

In [ ]:
#határozzuk meg, hogy mely boltok vannak a klasztereik közepéhez legközelebb
kmeans_centers = []
for k in range(5):
  min_distance = float("inf") #végtelen
  for i in range(len(dtf["Cluster Kmeans"])):
    if dtf.iloc[i, 9] == k:
      distance = np.sqrt((centers[k,0] - dtf.iloc[i,3])**2 + (centers[k,1] - dtf.iloc[i,4])**2)
      if distance < min_distance:
        min_distance = distance
        min_place = i
  kmeans_centers.append(min_place)
print(kmeans_centers)

In [ ]:
kmeans_iscentroid = []
for i in range(len(dtf)):
  if i not in kmeans_centers:
    kmeans_iscentroid.append(0)
  else:
    kmeans_iscentroid.append(1)
dtf["Kmeans Iscentroid"] = kmeans_iscentroid
dtf.head()